# Desafio (Parte 2) - Extraindo Informações

## Case baseado no desafio do Kaggle: "Instacart Market Basket Analysis"

Esse desafio é baseado em uma competição do Kaggle ([https://www.kaggle.com](https://www.kaggle.com)) de 2017, onde é pedido para fazer a predição de se um produto será comprado por dado usuário no carrinho atual, baseado em suas compras anteriores.

Não está no escopo da aula 



Link para o desafio: [https://www.kaggle.com/c/instacart-market-basket-analysis](https://www.kaggle.com/c/instacart-market-basket-analysis)

## Origem dos Dados:

Os dados mostrados nesse desafio são uma amostra extraída do dataset disponibilizado gratuitamente pela empresa Instacart ([https://www.instacart.com/](https://www.instacart.com/)) **\***. São dados reais do banco de dados da empresa, que foram anonimizados para o uso por pesquisadores e em competições de Data Science.

 
###### * “The Instacart Online Grocery Shopping Dataset 2017”, Accessed from [https://www.instacart.com/datasets/grocery-shopping-2017](https://www.instacart.com/datasets/grocery-shopping-2017) on March 10, 2017.

## Tabelas e Colunas Importantes

As Tabelas e suas respectivas colunas estão mostradas abaixo, junto com a descrição do significado de cada coluna:

* Colunas da Tabela `orders`:

    * ´order_id´: order identifier
    * ´user_id´: customer identifier
    * ´eval_set´: which evaluation set this order belongs in (see SET described below)
    * ´order_number´: the order sequence number for this user (1 = first, n = nth)
    * ´order_dow´: the day of the week the order was placed on
    * ´order_hour_of_day´: the hour of the day the order was placed on
    * ´days_since_prior´: days since the last order, capped at 30 (with NAs for order_number = 1)
    
* Colunas da Tabela `products (50k rows):

    * ´product_id´: product identifier
    * ´product_name´: name of the product
    * ´aisle_id´: foreign key
    * ´department_id´: foreign key

* Colunas da Tabela `aisles (134 rows):

    * ´aisle_id´: aisle identifier
    * ´aisle´: the name of the aisle

* Colunas da Tabela `deptartments (21 rows):

    * ´department_id´: department identifier
    * ´department´: the name of the department

* Colunas da Tabela `order_products__SET (30m+ rows):

    * ´order_id´: foreign key
    * ´product_id´: foreign key
    * ´add_to_cart_order´: order in which each product was added to cart
    * ´reordered´: 1 if this product has been ordered by this user in the past, 0 otherwise

___

# Import

In [55]:
import os
import numpy as np
import pandas as pd

# Objetivo:

Analisar o dataset gerado na Seção 3 e responder às questões.

# Dataset

In [56]:
file_name = os.path.join("data", "dataset.csv")

In [57]:
dataset = pd.read_csv(file_name, sep=",", index_col=0, encoding="utf-8")

C:\Users\tati\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [58]:
""" Facilitando a vida """
# baseado em uma thread do Kaggle sobre esse desafio, os dias da semana são definidos na seguinte ordem:
week_days = ["saturday", "sunday", "monday", "tuesday", "wednesday", "thursday", "friday"]

# traduzindo no dataset
dataset.order_dow = dataset.order_dow.map(lambda wd: week_days[wd])

In [59]:
dataset.head()

,order_id,user_id,order_dow,department,aisle,product_name
0,1187899,1,wednesday,beverages,soft drinks,Soda
1,2757217,67,saturday,beverages,soft drinks,Soda
2,632715,676,saturday,beverages,soft drinks,Soda
3,1167274,760,wednesday,beverages,soft drinks,Soda
4,3347074,804,tuesday,beverages,soft drinks,Soda


# Questões 

## A) Quantos usuários únicos fizeram pedidos?

In [60]:
""" Entre com o Código aqui """
len(dataset.user_id.unique())
#dataset.user_id.unique().shape[0]

117221

## B) Qual o produto que apareceu em mais pedidos?

In [44]:
""" Entre com o Código aqui """
dataset.product_name.value_counts()

Banana                                                16784
Bag of Organic Bananas                                13790
Organic Strawberries                                   9740
Organic Baby Spinach                                   8734
Large Lemon                                            7254
Organic Avocado                                        6642
Organic Hass Avocado                                   6545
Strawberries                                           5775
Limes                                                  5383
Organic Raspberries                                    5002
Organic Blueberries                                    4473
Organic Whole Milk                                     4384
Organic Cucumber                                       4122
Organic Zucchini                                       4108
Organic Yellow Onion                                   3790
Organic Garlic                                         3686
Seedless Red Grapes                     

In [61]:
g = dataset.groupby(["order_id", "product_name"])
product_counts = g.count().reset_index().set_index("order_id").product_name.value_counts()
print("Produto que ocorre em mais pedidos: {} ({} ocorrências)".format(product_counts.argmax(), product_counts.max()))

Produto que ocorre em mais pedidos: Banana (16784 ocorrências)


## C) Quais os usuários que fizeram mais e menos pedidos?

In [62]:
""" Entre com o Código aqui """
g = dataset.groupby(["user_id", "order_id"])
order_counts = g.count().reset_index().set_index("user_id").order_id.value_counts()
print("Usuário com mais pedidos: {} ({} ocorrências)".format(order_counts.argmax(), order_counts.max()))
print("Usuário com menos pedidos: {} ({} ocorrências)".format(order_counts.argmin(), order_counts.min()))

Usuário com mais pedidos: 2887678 (1 ocorrências)
Usuário com menos pedidos: 2887678 (1 ocorrências)


In [63]:
dataset.head()

,order_id,user_id,order_dow,department,aisle,product_name
0,1187899,1,wednesday,beverages,soft drinks,Soda
1,2757217,67,saturday,beverages,soft drinks,Soda
2,632715,676,saturday,beverages,soft drinks,Soda
3,1167274,760,wednesday,beverages,soft drinks,Soda
4,3347074,804,tuesday,beverages,soft drinks,Soda


In [70]:
bla = dataset[["user_id", "order_id"]].drop_duplicates(["user_id", "order_id"])
values = bla.user_id.value_counts()

print("Usuário com mais pedidos: {} ({} ocorrências)".format(values.argmax(), values.max()))
print("Usuário com menos pedidos: {} ({} ocorrências)".format(values.argmin(), values.min()))

Usuário com mais pedidos: 8188 (1 ocorrências)
Usuário com menos pedidos: 8188 (1 ocorrências)


## D) Qual a média e o desvio padrão da quantidade de produtos presentes em um pedido um usuário?

In [69]:
bla = dataset[["order_id","product_name"]]
values = bla.order_id.value_counts()
values.describe()[["mean", "std"]]

mean    10.564191
std      7.945047
Name: order_id, dtype: float64

In [68]:
g = dataset.groupby(["order_id"])
product_counts = g.product_name.count()
product_counts.describe()[["mean", "std"]]

mean    10.564191
std      7.945047
Name: product_name, dtype: float64

## E) Qual a média, a mediana, o mínimo e o máximo da quantidade de corredores (`aisles`) por departmento (`department`)?


In [71]:
dataset.department.unique().shape

(21,)

In [72]:
dataset.head()

,order_id,user_id,order_dow,department,aisle,product_name
0,1187899,1,wednesday,beverages,soft drinks,Soda
1,2757217,67,saturday,beverages,soft drinks,Soda
2,632715,676,saturday,beverages,soft drinks,Soda
3,1167274,760,wednesday,beverages,soft drinks,Soda
4,3347074,804,tuesday,beverages,soft drinks,Soda


In [76]:
bla = dataset[["department", "aisle"]].drop_duplicates(["department", "aisle"])
values = bla.department.value_counts()
values.describe()[["mean","50%","min","max"]]

mean     6.380952
50%      5.000000
min      1.000000
max     17.000000
Name: department, dtype: float64

In [73]:
""" Entre com o Código aqui """
aisles_per_dept = dataset[["department", "aisle"]].drop_duplicates(["department", "aisle"])
g = aisles_per_dept.groupby("department")
aisle_count = g.aisle.count()
aisle_count.describe()[["mean", "50%", "min", "max"]]

mean     6.380952
50%      5.000000
min      1.000000
max     17.000000
Name: aisle, dtype: float64

## F) Qual o dia da semana em que ocorrem mais pedidos de produtos para cada departmento (`department`)?

In [78]:
g = dataset.groupby(["department", "order_dow"])
product_count_per_dow = g.product_name.count()
product_count_per_dow_table = product_count_per_dow.unstack()
product_count_per_dow_table.head()

order_dow,friday,monday,saturday,sunday,thursday,tuesday,wednesday
department,,,,,,,
alcohol,683,578,640,571,929,753,853
babies,1769,1637,3143,1929,1761,1568,1517
bakery,6441,5081,10088,6338,5636,4799,4876
beverages,14623,12575,20249,15324,14215,12441,12578
breakfast,3694,3218,5936,4146,3434,3044,2984


In [79]:
product_count_per_dow_table.apply(lambda x: x.argmax(), axis=1)

department
alcohol            thursday
babies             saturday
bakery             saturday
beverages          saturday
breakfast          saturday
bulk               saturday
canned goods       saturday
dairy eggs         saturday
deli               saturday
dry goods pasta    saturday
frozen             saturday
household          saturday
international      saturday
meat seafood       saturday
missing            saturday
other              saturday
pantry             saturday
personal care      saturday
pets               saturday
produce            saturday
snacks             saturday
dtype: object

##  G) Quais foram os 5% piores e os 5% melhores corredores (`aisle`) em termos de vendas? 

In [80]:
""" Entre com o Código aqui """
aisles_sales_count = dataset.groupby("aisle").product_name.count()
l_inf, l_sup = aisles_sales_count.describe(percentiles=[.05, .95])[["5%", "95%"]]
l_inf, l_sup

(458.44999999999999, 28386.649999999991)

In [81]:
"""piores corredores"""
aisles_sales_count[aisles_sales_count < l_inf].sort_values()

aisle
beauty                        247
frozen juice                  251
baby accessories              273
baby bath body care           286
kitchen supplies              410
specialty wines champagnes    416
ice cream toppings            450
Name: product_name, dtype: int64

In [82]:
"""melhores corredores"""
aisles_sales_count[aisles_sales_count > l_sup].sort_values(ascending=False)

aisle
fresh fruits                     134576
fresh vegetables                 134559
packaged vegetables fruits        70176
yogurt                            49372
packaged cheese                   37390
water seltzer sparkling water     32647
milk                              29192
Name: product_name, dtype: int64

##  H) Quais foram os 5 produtos mais comprados de cada departamento na terça-feira? 

### Mostre em uma tabela como no exemplo:

| departamento | 1o produto | 2o produto | 3o produto | 4o produto | 5o produto |
|:------------ |:---------- |:---------- |:---------- |:---------- |:---------- |
| depart 01    | prod A     | prod W     | prod L     | prod H     | prod D     |
| depart 02    | prod X     | prod Q     | prod B     | prod S     | prod P     |
| ...          | ...        | ...        | ...        | ...        | ...        |


In [84]:
""" Entre com o Código aqui """
tuesday_orders = dataset[dataset.order_dow == "tuesday"]
depart_prod_count = pd.crosstab(tuesday_orders.department, tuesday_orders.product_name)

In [96]:
depart_prod_count.head()

product_name,#2 Coffee Filters,#2 Cone White Coffee Filters,0 Calorie Fuji Apple Pear Water Beverage,0 Calorie Strawberry Dragonfruit Water Beverage,0% Fat Blueberry Greek Yogurt,0% Fat Free Organic Milk,0% Fat Organic Greek Vanilla Yogurt,0% Fat Strawberry Greek Yogurt,0% Fat Superfruits Greek Yogurt,0% Fat Vanilla Greek Yogurt,...,of Hanover 100 Calorie Pretzels Mini,smartwater® Electrolyte Enhanced Water,vitaminwater® XXX Acai Blueberry Pomegranate,with Crispy Almonds Cereal,with Dawn Action Pacs Fresh Scent Dishwasher Detergent Pacs,with Olive Oil Mayonnaise,with Olive Oil Mayonnaise Dressing,with Xylitol Cinnamon 18 Sticks Sugar Free Gum,with Xylitol Watermelon Twist 18 Sticks Sugar Free Gum,with a Splash of Mango Coconut Water
department,,,,,,,,,,,,,,,,,,,,,
alcohol,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
babies,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bakery,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
beverages,1,1,2,1,0,0,0,0,0,0,...,0,27,18,0,0,0,0,0,0,1
breakfast,0,0,0,0,0,0,0,0,0,0,...,0,0,0,21,0,0,0,0,0,0


In [98]:
bla = depart_prod_count.T
bla.head()

department,alcohol,babies,bakery,beverages,breakfast,bulk,canned goods,dairy eggs,deli,dry goods pasta,...,household,international,meat seafood,missing,other,pantry,personal care,pets,produce,snacks
product_name,,,,,,,,,,,,,,,,,,,,,
#2 Coffee Filters,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#2 Cone White Coffee Filters,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0 Calorie Fuji Apple Pear Water Beverage,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0 Calorie Strawberry Dragonfruit Water Beverage,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0% Fat Blueberry Greek Yogurt,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
top_5_prods_per_dept = bla.apply(lambda x: x.sort_values(ascending=False)[:5].index.tolist(), axis=0)
top_5_prods_per_dept.head()

department
alcohol      [Cabernet Sauvignon, Sauvignon Blanc, India Pa...
babies       [Baby Food Stage 2 Blueberry Pear & Purple Car...
bakery       [100% Whole Wheat Bread, Organic Bread with 21...
beverages    [Sparkling Water Grapefruit, Spring Water, Sod...
breakfast    [Honey Nut Cheerios, Organic Old Fashioned Rol...
dtype: object

In [100]:
top_5_prods_per_dept = depart_prod_count.apply(lambda x: x.sort_values(ascending=False)[:5].index.tolist(), axis=1)
top_5_prods_per_dept.head()

department
alcohol      [Cabernet Sauvignon, Sauvignon Blanc, India Pa...
babies       [Baby Food Stage 2 Blueberry Pear & Purple Car...
bakery       [100% Whole Wheat Bread, Organic Bread with 21...
beverages    [Sparkling Water Grapefruit, Spring Water, Sod...
breakfast    [Honey Nut Cheerios, Organic Old Fashioned Rol...
dtype: object

In [101]:
for dept in top_5_prods_per_dept.index:
    print("Department '{}':".format(dept))
    print(" - " + "\n - ".join(top_5_prods_per_dept.loc[dept]))

Department 'alcohol':
 - Cabernet Sauvignon
 - Sauvignon Blanc
 - India Pale Ale
 - Beer
 - Chardonnay
Department 'babies':
 - Baby Food Stage 2 Blueberry Pear & Purple Carrot
 - Gluten Free SpongeBob Spinach Littles
 - Spinach Peas & Pear Stage 2 Baby Food
 - Peach,  Apricot & Banana Stage 2 Baby Food
 - Broccoli & Apple Stage 2 Baby Food
Department 'bakery':
 - 100% Whole Wheat Bread
 - Organic Bread with 21 Whole Grains
 - Ezekiel 4:9 Bread Organic Sprouted Whole Grain
 - Organic 21 Grain Thin Sliced Bread
 - Original Nooks & Crannies English Muffins
Department 'beverages':
 - Sparkling Water Grapefruit
 - Spring Water
 - Soda
 - Sparkling Natural Mineral Water
 - Lime Sparkling Water
Department 'breakfast':
 - Honey Nut Cheerios
 - Organic Old Fashioned Rolled Oats
 - Raisin Bran Cereal
 - Cereal
 - Cherrios Honey Nut
Department 'bulk':
 - Dried Mango
 - Organic Rolled Oats
 - Organic Black Mission Figs
 - Apricots
 - Vegetable Chips
Department 'canned goods':
 - Organic Black Bean

## I) Quais são os 2 corredores (`aisles`) que são visitados juntos no mesmo pedido mais vezes em uma sexta-feira?

In [105]:
""" Entre com o Código aqui """
friday_orders = dataset[dataset.order_dow == "friday"]
aisles_visit_count = pd.crosstab(friday_orders.order_id, friday_orders.aisle)
aisles_visit_count.head()

aisle,air fresheners candles,asian foods,baby accessories,baby bath body care,baby food formula,bakery desserts,baking ingredients,baking supplies decor,beauty,beers coolers,...,spreads,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt
order_id,,,,,,,,,,,,,,,,,,,,,
36,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
393,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
473,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
aisles_visit_count = aisles_visit_count.applymap(lambda x: 1 if x > 0 else 0)

In [107]:
co_occurrent_aisles = aisles_visit_count.T

In [108]:
co_occurrent_aisles = co_occurrent_aisles.dot(aisles_visit_count)
co_occurrent_aisles.head()

aisle,air fresheners candles,asian foods,baby accessories,baby bath body care,baby food formula,bakery desserts,baking ingredients,baking supplies decor,beauty,beers coolers,...,spreads,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt
aisle,,,,,,,,,,,,,,,,,,,,,
air fresheners candles,129,9,0,1,5,2,8,0,0,0,...,9,11,2,8,0,7,2,41,2,28
asian foods,9,805,2,3,34,8,103,12,2,6,...,92,51,78,80,8,7,20,181,6,244
baby accessories,0,2,39,2,7,0,5,1,0,0,...,5,2,2,4,1,1,6,13,0,19
baby bath body care,1,3,2,40,7,1,2,0,0,1,...,5,3,0,4,1,2,4,11,0,17
baby food formula,5,34,7,7,615,8,67,2,2,2,...,83,33,33,45,9,8,11,156,4,297


In [110]:
co_occurrent_aisles -= np.triu(co_occurrent_aisles)
co_occurrent_aisles.head()

aisle,air fresheners candles,asian foods,baby accessories,baby bath body care,baby food formula,bakery desserts,baking ingredients,baking supplies decor,beauty,beers coolers,...,spreads,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt
aisle,,,,,,,,,,,,,,,,,,,,,
air fresheners candles,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
asian foods,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
baby accessories,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
baby bath body care,1,3,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
baby food formula,5,34,7,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [113]:
co_occurrent_aisles.stack().sort_values(ascending=False).head(1)

aisle             aisle       
fresh vegetables  fresh fruits    5809
dtype: int64

## J) Quais as duplas de produtos que mais saem juntas no pedido (`order_id`) de um usuário em uma sexta-feira?

In [ ]:
""" Entre com o Código aqui """
%time
friday_orders = dataset[dataset.order_dow == "friday"]
products_count = (pd.crosstab(friday_orders.order_id, friday_orders.product_name) > 0).astype(float)
co_occurrent_products = products_count.T.dot(products_count)
co_occurrent_products -= np.triu(co_occurrent_products)

Wall time: 2 ms


In [ ]:
co_occurrent_products.stack().sort_values(ascending=False).head(1)